In [ ]:
from neo4j import GraphDatabase # FOR AIRFLOW


NEO4J_USER = "neo4j"
NEO4J_URI = "bolt://172.17.0.2:7687" ## ip drom docker 
NEO4J_PASSWORD = "internet"

### IF AIRFLOW NOT WORKING, USE THIS SAVE FINAL FILE TO AIRFLOW import folder and Load from there or SETUP the localhost situation like in graph_db.ipynb
#neo4j_driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
graph = GraphDatabase.driver("bolt://localhost:11005", auth=("neo4j", "internet_memes"))

with graph.session(database="neo4j") as session:


    try:
        cypher_query = "Match () Return 1 Limit 1"
        result = session.run(cypher_query)
        print("ok")
    except: 
        print("not ok")


not ok


The NEO4J run on a locally set up server 

In [6]:
from py2neo import Graph

graph = Graph("bolt://localhost:11005", auth=("neo4j", "internet_memes"))
## checking if the server is up and running 
try:
    graph.run("Match () Return 1 Limit 1")
    print('ok')
except Exception:
    print('not ok')


ok


In [7]:
import pandas as pd 
import numpy as np
# reading it in from cleaned data
main_df = pd.read_csv("/home/marilin/Documents/Data engineering/data_project/main.csv", encoding='utf-8', index_col=False)
main_df = main_df.dropna()
del main_df["description"]
lst = []
for link in list(main_df["url"]):
    lst.append(link.split("/")[-1])
main_df["meme_name"] = lst

main_df.to_csv("/home/marilin/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-fc4db720-e397-4b7a-a229-692d78772748/import/main.csv", encoding='utf-8', index=False)

main_df.head()


,title,url,views,author,updater,added,updated,year,has_relations,has_other_texts,...,has_DBpedia,status,adult,medical,racy,spoof,violence,template_image_url,fb_image_url,meme_name
0,This is Relevant To My Interests,https://knowyourmeme.com/memes/this-is-relevan...,136103,jamie-dubs,sophie--8,2008-12-12,2019-01-09,2006,False,True,...,False,confirmed,VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,VERY_LIKELY,UNLIKELY,https://i.kym-cdn.com/entries/icons/original/0...,https://i.kym-cdn.com/entries/icons/facebook/0...,this-is-relevant-to-my-interests
1,ROFLcopter,https://knowyourmeme.com/memes/roflcopter,537218,jamie-dubs,13acab12,2008-12-12,2020-06-06,2004,True,True,...,True,confirmed,VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,POSSIBLE,VERY_UNLIKELY,https://i.kym-cdn.com/entries/icons/original/0...,https://i.kym-cdn.com/entries/icons/facebook/0...,roflcopter
2,Leave Britney Alone,https://knowyourmeme.com/memes/leave-britney-a...,211104,jamie-dubs,andcallmeshirley,2008-12-12,2021-02-12,2007,False,True,...,True,confirmed,UNLIKELY,UNLIKELY,UNLIKELY,LIKELY,UNLIKELY,https://i.kym-cdn.com/entries/icons/original/0...,https://i.kym-cdn.com/entries/icons/facebook/0...,leave-britney-alone
3,O RLY?,https://knowyourmeme.com/memes/o-rly,1069521,longcat,andcallmeshirley,2008-12-12,2021-11-22,2003,True,True,...,True,confirmed,VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY,VERY_LIKELY,POSSIBLE,https://i.kym-cdn.com/entries/icons/original/0...,https://i.kym-cdn.com/entries/icons/facebook/0...,o-rly
4,Yatta!,https://knowyourmeme.com/memes/yatta,41057,toddpersen,twenty-one,2009-02-18,2018-07-18,2001,False,True,...,True,confirmed,POSSIBLE,VERY_UNLIKELY,POSSIBLE,VERY_UNLIKELY,VERY_UNLIKELY,https://i.kym-cdn.com/entries/icons/original/0...,https://i.kym-cdn.com/entries/icons/facebook/0...,yatta


In [9]:
import pandas as pd 
relations_df = pd.read_csv("/home/marilin/Documents/Data engineering/data_project/relations.csv", encoding='utf-8') 
relations_df = relations_df.rename(columns={'Unnamed: 0': 'meme_name'})
# replacing nan with unknown
relations_df = relations_df.fillna("unknown")
del relations_df["children"]

memes = relations_df["meme_name"].tolist()
siblings = relations_df["siblings"].tolist()
parents = relations_df["parent"].tolist()

meme_col, sib_col, p_col = [], [], []

for i, meme in enumerate(memes):

    meme_labels = siblings[i].strip("[]'").split("', '")


    for j in range(min(5, len(meme_labels))):
        p_col.append(parents[i])
        meme_col.append(meme)
        sib_col.append(meme_labels[j])


mfa = pd.DataFrame()
mfa["meme_name"] = meme_col
mfa["parent"] = p_col
mfa["siblings"] = sib_col

mfa.head()
mfa.to_csv("/home/marilin/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-fc4db720-e397-4b7a-a229-692d78772748/import/relations.csv", encoding='utf-8', index=False)

In [15]:
mfa.head()

,meme_name,parent,siblings
0,roflcopter,lol-ascii,lollerskates
1,roflcopter,lol-ascii,roflcopter
2,o-rly,unknown,an-hero
3,o-rly,unknown,butthurt
4,o-rly,unknown,do-it-faggot


In [10]:
imgrec_df = pd.read_csv("/home/marilin/Documents/Data engineering/data_project/img_rec.csv", encoding='utf-8')
imgrec_df = imgrec_df.rename(columns={'Unnamed: 0': 'meme_name'})
imgrec_df = imgrec_df.fillna("unknown")

memes = imgrec_df["meme_name"].tolist()
labels = imgrec_df['image_recogn_labels'].tolist()

meme_col, label_col = [], []

for i, meme in enumerate(memes):
    meme_labels = labels[i].strip("[]'").split("', '")

    for j in range(min(5, len(meme_labels))):
        meme_col.append(meme)
        label_col.append(meme_labels[j])


mf = pd.DataFrame()
mf["meme_name"] = meme_col
mf["image_recogn_labels"] = label_col

mf.to_csv("/home/marilin/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-fc4db720-e397-4b7a-a229-692d78772748/import/img_rec.csv", encoding='utf-8', index=False)

In [11]:
query_main = """
LOAD CSV WITH HEADERS FROM "file:///main.csv" AS csvLine 
MERGE (m:Meme_name {meme_name:csvLine.meme_name, url:csvLine.url, views:toInteger(csvLine.views)})
MERGE (a:Meme_author {name: csvLine.author})
MERGE (r:Recent_update {updated: csvLine.updated})
MERGE (t:Meme_template {template_image: csvLine.template_image_url})
CREATE (m)-[:IS_CREATED_BY]->(a), (m)-[:BASED_ON]->(t), (m)-[:RECENTLY_UPDATED_ON]->(r)
"""

In [12]:
graph.run(query_main)

(No data)

In [16]:
query_relations = """
LOAD CSV WITH HEADERS FROM "file:///relations.csv" AS csvLine
MERGE (p:Parent {parent_name: csvLine.parent})
MERGE (s:Siblings {siblings: csvLine.siblings})
MERGE (m:Meme_name {meme_name:csvLine.meme_name})
CREATE (m)-[:IS_CHILD_OF]->(p), (m)-[:IS_RELATED_TO]->(s);
"""

In [17]:
graph.run(query_relations)

(No data)

In [18]:
query_img = """
LOAD CSV WITH HEADERS FROM "file:///img_rec.csv" AS csvLine
MERGE (m:Meme_name {meme_name:csvLine.meme_name})
MERGE (i: image_recogn_labels{image_label_name:csvLine.image_recogn_labels})
CREATE (m)-[:CONTAINS]->(i);
"""

In [19]:
graph.run(query_img)

In [118]:
#### THIS IS THE END

sim_df = pd.read_csv("/home/marilin/Documents/Data engineering/data_project/sim_img.csv", encoding='utf-8')
sim_df = sim_df.rename(columns={'Unnamed: 0': 'meme_name'})
sim_df = sim_df.fillna("unknown")
# saving to neo4j local folder 
sim_df.to_csv("/home/marilin/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-fc4db720-e397-4b7a-a229-692d78772748/import/sim_img.csv", encoding='utf-8', index=False)

In [119]:
sim_df.head()

,meme_name,similar_images_all
0,this-is-relevant-to-my-interests,['https://i.kym-cdn.com/photos/images/newsfeed...
1,roflcopter,['https://i.kym-cdn.com/photos/images/newsfeed...
2,leave-britney-alone,unknown
3,o-rly,['https://i.kym-cdn.com/photos/images/newsfeed...
4,yatta,unknown


In [124]:
query_sim = """
LOAD CSV WITH HEADERS FROM "file:///sim_img.csv" AS csvLine 

MERGE (p:Similar_images {similar_images: csvLine.similar_images_all})
MERGE (m:Meme_name {meme_name:csvLine.meme_name})
CREATE (m)-[:IS_SIMILAR_TO_FOREIGN_LINK]->(p);
"""

In [125]:
graph.run(query_sim)

(No data)

In [127]:
vis_df = pd.read_csv("/home/marilin/Documents/Data engineering/data_project/vision.csv", encoding='utf-8')
vis_df = vis_df.rename(columns={'Unnamed: 0': 'meme_name'})
vis_df = vis_df.fillna("unknown")
vis_df.to_csv("/home/marilin/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-fc4db720-e397-4b7a-a229-692d78772748/import/vision.csv", encoding='utf-8', index=False)
vis_df.head()

,meme_name,memes,scores
0,Health care provider,['thats-wicked-%E3%83%9E%E3%82%B8%E3%82%84%E3%...,"['0.93678385, 0.5965857']"
1,Halictidae,['petty-kobe-bryant'],['0.6376406']
2,Squeezebox,['poorfag-chan'],['0.8893292500000001']
3,Sitting,"['zergface', 'zac-efrons-baywatch-faceplant', ...","['0.71677357, 0.71657026, 0.715113, 0.71499039..."
4,Python,['beta-uprising'],['0.7320513']
